In [7]:
import cv2
import base64
import requests
import time
import numpy as np

# Load video
video_path = "Test1.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("❌ Failed to open video. Check the path.")
else:
    print("✅ Video opened successfully. Starting processing...")


# Prepare to write the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_with_kalman.mp4', fourcc, 20.0, 
                      (int(cap.get(3)), int(cap.get(4))))

# Roboflow API endpoint with lowered confidence threshold
API_URL = "https://detect.roboflow.com/nfl-object-tracking/4?api_key=RsIHVYfJftsspBQBQPkK&confidence=0.2"

frame_num = 0

# Setup Kalman Filter
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

predicted = None


while True:
    ret, frame = cap.read()
    if not ret:
        print("✅ End of video or failed to read frame.")
        break
    print(f"▶️ Processing frame {frame_num}")
    # Encode frame as JPEG
    _, buffer = cv2.imencode('.jpg', frame)
    b64 = base64.b64encode(buffer).decode('utf-8')

    # Send frame to Roboflow
    response = requests.post(API_URL, data=b64, headers={"Content-Type": "application/x-www-form-urlencoded"})
    
    try:
        predictions = response.json().get("predictions", [])
    except:
        predictions = []

    detected = False
    for pred in predictions:
        if pred["class"] == "Ball" and float(pred["confidence"]) > 0.2:
            x, y = int(pred["x"]), int(pred["y"])
            detected = True

            # Kalman correct
            measurement = np.array([[np.float32(x)], [np.float32(y)]])
            kalman.correct(measurement)

            # Draw detection
            w, h = int(pred["width"]), int(pred["height"])
            x1 = x - w // 2
            y1 = y - h // 2
            x2 = x + w // 2
            y2 = y + h // 2
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ball ({pred['confidence']:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            break

    if not detected:
        predicted = kalman.predict()
        px, py = int(predicted[0]), int(predicted[1])
        # Draw predicted point
        cv2.circle(frame, (px, py), 10, (0, 0, 255), 2)
        cv2.putText(frame, "Predicted", (px + 15, py),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Save the annotated frame to output video
    out.write(frame)

    frame_num += 1
    time.sleep(0.1)  # API rate safety delay

# Cleanup
cap.release()
out.release()

✅ Video opened successfully. Starting processing...
▶️ Processing frame 0


C:\Users\fjwhe\AppData\Local\Temp\ipykernel_4440\264794906.py:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  px, py = int(predicted[0]), int(predicted[1])


▶️ Processing frame 1
▶️ Processing frame 2
▶️ Processing frame 3
▶️ Processing frame 4
▶️ Processing frame 5
▶️ Processing frame 6
▶️ Processing frame 7
▶️ Processing frame 8
▶️ Processing frame 9
▶️ Processing frame 10
▶️ Processing frame 11
▶️ Processing frame 12
▶️ Processing frame 13
▶️ Processing frame 14
▶️ Processing frame 15
▶️ Processing frame 16
▶️ Processing frame 17
▶️ Processing frame 18
▶️ Processing frame 19
▶️ Processing frame 20
▶️ Processing frame 21
▶️ Processing frame 22
▶️ Processing frame 23
▶️ Processing frame 24
▶️ Processing frame 25
▶️ Processing frame 26
▶️ Processing frame 27
▶️ Processing frame 28
▶️ Processing frame 29
▶️ Processing frame 30
▶️ Processing frame 31
▶️ Processing frame 32
▶️ Processing frame 33
▶️ Processing frame 34
▶️ Processing frame 35
▶️ Processing frame 36
▶️ Processing frame 37
▶️ Processing frame 38
▶️ Processing frame 39
▶️ Processing frame 40
▶️ Processing frame 41
▶️ Processing frame 42
▶️ Processing frame 43
▶️ Processing frame 